# Устанавливаем всё необходимое

In [189]:
#!g1.1
%pip install transformers
%pip install emoji
%pip install google_trans_new

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.


# Код для поддержки бота

In [190]:
#!g1.1
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
class DialoGPT:
    tokenizer = AutoTokenizer.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2")
    model = AutoModelForCausalLM.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2")
    chat_history = {}

    def get_length_param(self, text: str) -> str:
        tokens_count = len(self.tokenizer.encode(text))
        if tokens_count <= 15:
            len_param = '1'
        elif tokens_count <= 50:
            len_param = '2'
        elif tokens_count <= 256:
            len_param = '3'
        else:
            len_param = '-'
        return len_param

    """
    Возвращает ответ на текст, введённый пользователем
    @param input_user Текст, введённый пользователе
    @param chat_id id чата, куда был введен ответ или -1, если чата нет
    @returns Ответ на текст, введённый пользователем
    """
    def get_response(self, input_user, chat_id):
        input_user = input_user[:256]
        tokenizer = self.tokenizer
        
        # encode the new user input, add parameters and return a tensor in Pytorch
        new_user_input = f"|0|{self.get_length_param(input_user)}|" + input_user + tokenizer.eos_token +  "|1|1|"
        new_user_input_ids = tokenizer.encode(new_user_input, return_tensors="pt")

        # append the new user input tokens to the chat history
        bot_input_ids = torch.cat([self.chat_history[chat_id], new_user_input_ids], dim=-1) if chat_id in self.chat_history else new_user_input_ids

        # generated a response
        self.chat_history[chat_id] = self.model.generate(
            bot_input_ids,
            num_return_sequences=1,
            max_length=512,
            no_repeat_ngram_size=3,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature = 0.6,
            mask_token_id=tokenizer.mask_token_id,
            eos_token_id=tokenizer.eos_token_id,
            unk_token_id=tokenizer.unk_token_id,
            pad_token_id=tokenizer.pad_token_id,
            device='cuda',
        )
        print(len(self.chat_history[chat_id]))
        
        result = tokenizer.decode(self.chat_history[chat_id][:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
        if chat_id == -1:
            self.restart(-1)
        
        return result
    
    def restart(self, chat_id):
        if chat_id in self.chat_history:
            del self.chat_history[chat_id]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Проверяем, что всё ок

In [ ]:
#!g1.1
from .utils.telegram_utils import *

[Errno 2] No such file or directory: '/home/jupyter/work/resources/MakeUHappyChatBot/prod/telegram'
Traceback (most recent call last):
  File "/kernel/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-4cd773378f3d>", line 1, in <module>
    from utils.telegram_utils import *
ModuleNotFoundError: No module named 'utils'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/kernel/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ModuleNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/kernel/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1170, in get_records
    return _fixed_getinnerframe

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Internal error. Results of this cell may be lost. Please contact support.
ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TypeError("object of type 'NoneType' has no len()")>
Traceback (most recent call last):
  File "/kernel/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-4cd773378f3d>", line 1, in <module>
    from utils.telegram_utils import *
ModuleNotFoundError: No module named 'utils'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/kernel/lib/python3.

In [ ]:
#!/bin/bash

In [ ]:
#!g1.1
get_text('привет')

In [191]:
#!g1.1
chat_bot = DialoGPT()
chat_bot.get_response('привет', -1)

1


'Привет, ты кто?'

# Основной цикл работы бота

In [195]:
#!g1.1
from ...utils.telegram_utils import *

def work():
    get_url = 'https://api.telegram.org/bot1853926901:AAHs-iYO-hp559x52L67egfEZ8FzLo26WAk/getUpdates?offset=' + str(offset)
    result = requests.get(get_url).json()

    if len(result['result']) == 0:
        return
    #print(result)

    result = result['result'][0]
    if 'message' in result:
        message = result['message']
        chat_id = message['chat']['id']

        text = get_text(message)
        print(text)
        text, should_send = process_command(text, chat_id)
        
        if should_send:
            text = chat_bot.get_response(text, chat_id)

        if len(text) > 0:
            send_url = 'https://api.telegram.org/bot1853926901:AAHs-iYO-hp559x52L67egfEZ8FzLo26WAk/sendMessage?chat_id=' \
                + str(chat_id) + '&text=' + text 
            requests.get(send_url)
    offset = int(result['update_id']) + 1
    offset_file.truncate()
    offset_file.write(str(offset))

Execute error: Failed to allocate servant g1.1: internal error. Please try again

In [171]:
#!g1.1
import requests

offset_file = open('offset.txt', 'r+')
offset = int(offset_file.read())
print('Started')
while True:
    try:
        work()
    except KeyboardInterrupt:
        print('Stopped')

Started
ghbdtn
привет
давай говорить на русском
и что?
привет
Серьезное лицо с символами, покрывающими рот


KeyboardInterrupt: 

In [105]:
#!g1.1
import pickle
d = {
    'a':5,
    'b':1
}

f = open('test', 'wb')
pickle.dump(d, f)

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:815: UserWarning: The following variables cannot be serialized: f
  warnings.warn(message)


In [ ]:
#!g1.1
